In [1]:
import tensorflow as tf
import cv2 as cv
import os
import pandas as pd
import shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from math import floor, ceil, pi
import scipy.misc
import glob
import imageio # for saving image file
import tensorflow_addons as tfa # for rotating image

In [13]:
def get_image_paths():
    folder = "../img_preparation/gauges"
    files = os.listdir(folder)
    files.sort()
    files = ["{}/{}".format(folder, file) for  file in files]
    return files

In [3]:
os.getcwd()

'/Users/paramatephuengtrakul/project2022/computer_vision/Gauge_classification/regression_task/img_preparation'

In [17]:
for dirpath, dirnames, filenames in os.walk("../img_preparation/gauges"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath} ")

There are 0 directories and 41 images in ../img_preparation/gauges 


In [18]:
X_img_paths = get_image_paths()
print(X_img_paths[:])

['../img_preparation/gauges/gauge_-0.0.png', '../img_preparation/gauges/gauge_-0.1.png', '../img_preparation/gauges/gauge_-0.2.png', '../img_preparation/gauges/gauge_-0.3.png', '../img_preparation/gauges/gauge_-0.4.png', '../img_preparation/gauges/gauge_-0.5.png', '../img_preparation/gauges/gauge_-0.6.png', '../img_preparation/gauges/gauge_-0.7.png', '../img_preparation/gauges/gauge_-0.8.png', '../img_preparation/gauges/gauge_-0.9.png', '../img_preparation/gauges/gauge_-1.0.png', '../img_preparation/gauges/gauge_0.1.png', '../img_preparation/gauges/gauge_0.2.png', '../img_preparation/gauges/gauge_0.3.png', '../img_preparation/gauges/gauge_0.4.png', '../img_preparation/gauges/gauge_0.5.png', '../img_preparation/gauges/gauge_0.6.png', '../img_preparation/gauges/gauge_0.7.png', '../img_preparation/gauges/gauge_0.8.png', '../img_preparation/gauges/gauge_0.9.png', '../img_preparation/gauges/gauge_1.0.png', '../img_preparation/gauges/gauge_1.1.png', '../img_preparation/gauges/gauge_1.2.png',

In [19]:
def tf_resize_img(X_img_file_paths, img_shape=384):
    X_data = []

# Individually resize image
    for index, file_path in enumerate(X_img_file_paths):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, size=[img_shape, img_shape])
        img = img/225.
        X_data.append(img)

    X_data = np.array(X_data, dtype=np.float32)
    return X_data

In [20]:
X_imgs = tf_resize_img(X_img_paths[:])
print(X_imgs.shape)

(41, 384, 384, 3)


## Scaling

In [21]:
def central_scale_images(X_imgs, scales, img_shape=384):
    boxes = np.zeros((len(scales), 4), dtype=np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype=np.float32)
    box_ind = np.zeros(len(scales), dtype=np.int32)
    crop_size = np.array([img_shape, img_shape], dtype=np.int32)

    X_scale_data= []
    for img_data in X_imgs:
        batch_img = tf.expand_dims(img_data, axis=0) # expand dimension of input image to 4 dims
        tf_img = tf.image.crop_and_resize(batch_img, boxes, box_ind, crop_size) # require 4 dims input
        X_scale_data.extend(tf_img)

    X_scale_data = np.array(X_scale_data, dtype=np.float32)
    return X_scale_data

# Translation

In [22]:
IMAGE_SIZE = 384

In [23]:
def get_translate_parameter(index):
    if index == 0: # Translate left 20 percent
        offset = np.array([0.0, 0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = 0
        w_end = int(ceil(0.8 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 1: # Translate right 20 percent
        offset = np.array([0.0, -0.2], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.8 * IMAGE_SIZE)], dtype = np.int32)
        w_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 2: # Translate top 20 percent
        offset = np.array([0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(ceil(0.8 * IMAGE_SIZE)) 
    elif index == 3: # Translate bottom 20 percent
        offset = np.array([-0.2, 0.0], dtype = np.float32)
        size = np.array([ceil(0.8 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(floor((1 - 0.8) * IMAGE_SIZE))
        h_end = IMAGE_SIZE
    elif index == 4: # Translate left 10 percent
        offset = np.array([0.0, 0.1], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.9 * IMAGE_SIZE)], dtype = np.int32)
        w_start = 0
        w_end = int(ceil(0.9 * IMAGE_SIZE))
        h_start = 0
        h_end = IMAGE_SIZE
    elif index == 5: # Translate right 10 percent
        offset = np.array([0.0, -0.1], dtype = np.float32)
        size = np.array([IMAGE_SIZE, ceil(0.9 * IMAGE_SIZE)], dtype = np.int32)
        w_start = int(floor((1 - 0.9) * IMAGE_SIZE))
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = IMAGE_SIZE 
    elif index == 6: # Translate top 10 percent
        offset = np.array([0.1, 0.0], dtype = np.float32)
        size = np.array([ceil(0.9 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = 0
        h_end = int(ceil(0.9 * IMAGE_SIZE))  
    elif index == 7: # Translate bottom 10 percent
        offset = np.array([-0.1, 0.0], dtype = np.float32)
        size = np.array([ceil(0.9 * IMAGE_SIZE), IMAGE_SIZE], dtype = np.int32)
        w_start = 0
        w_end = IMAGE_SIZE
        h_start = int(floor((1 - 0.9) * IMAGE_SIZE))
        h_end = IMAGE_SIZE      
        
    return offset, size, w_start, w_end, h_start, h_end

def translate_images(X_imgs):
    offsets = np.zeros((len(X_imgs), 2), dtype=np.float32)
    n_translates=8
    X_translated_arr = []

    for i in range(n_translates):
        X_translated = np.zeros((len(X_imgs), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        X_translated.fill(1.0) # filling background color
        base_offset, size, w_start, w_end, h_start, h_end = get_translate_parameter(i)
        offsets[:, :] = base_offset
        glimpses = tf.image.extract_glimpse(X_imgs, size, offsets)
        X_translated[:, h_start: h_start + size[0],
                     w_start: w_start + size[1], :] = glimpses
        X_translated_arr.extend(X_translated)
    X_translated_arr = np.array(X_translated_arr, dtype=np.float32)
    return X_translated_arr

## Rotation

In [24]:
def rotate_images(X_imgs, start_angle, end_angle, n_images):
    X_rotate = []
    iterate_at = (end_angle - start_angle) / (n_images - 1)
    for index in range(n_images):
        degree_angle = start_angle + index * iterate_at
        radian_value = degree_angle * pi / 180 # convert  to radian
        radian_arr = [radian_value] * len(X_imgs)
        rotated_imgs = tfa.image.rotate(X_imgs, radian_arr)
        X_rotate.extend(rotated_imgs)

    X_rotate = np.array(X_rotate, dtype=np.float32)
    return X_rotate 

## Light Condition

In [25]:
def add_gaussian_noise(X_imgs):
    gaussian_noise_imgs = []
    row, col, _ = X_imgs[0].shape
    count=0
    # Gaussian distribution paramaters
    mean = 0
    var = 0.1
    sigma = var ** 0.5

    # parameters of method cv.addWeighted
    ALPHA = np.linspace(0.50, 0.85, 2)
    BETA = np.linspace(0.2, 0.8, 2)

    for X_img in X_imgs:
        for alpha in ALPHA:
            for beta in BETA:
                gaussian = np.random.random((row, col, 1)).astype(np.float32)
                gaussian = np.concatenate((gaussian, gaussian, gaussian), axis=2)
                gaussian_img = cv.addWeighted(X_img, alpha, (1-alpha) * gaussian, beta, 0)
                gaussian_noise_imgs.append(gaussian_img)
                

    gaussian_noise_imgs = np.array(gaussian_noise_imgs, dtype=np.float32)
    return gaussian_noise_imgs

In [31]:
# os.mkdir("img_augment_384")

In [32]:
filename1 = "gauge_0.png"
path = "../img_preparation/tf_gauges/psi_{0}/{1}"
save_path = "../img_preparation/img_augment_384/{0}"

In [37]:
import os, glob
 
dir = "../img_preparation/img_augment_384/"
for file in os.scandir(dir):
    os.remove(file.path)

In [38]:
values = []

flag = True
count = 0

while flag:
    for j in np.arange(-1, 3+0.1, 0.1):
        scale_num = "{:.1f}".format(j)
        int_scale = float(scale_num)
        X_img = path.format(scale_num, filename1)
        X_resize = tf_resize_img([X_img])
        count+=1

        # Save Resized image
        values.append(int_scale)
        file_resized = "{0}.jpg".format(count)
        filename_resized = save_path.format(file_resized)
        imageio.imwrite(filename_resized, X_resize[0])

        # Scaling
        scale = np.linspace(0.6, 0.9, 28)
        scaled_imgs = central_scale_images(X_resize, scale)

        for i in range(0, len(scale)):
            count+=1
            values.append(int_scale)
            filename = "{0}.jpg".format(count)
            filepath = save_path.format(filename)
            imageio.imwrite(filepath, scaled_imgs[i])

        # Translation
        translated_imgs = translate_images(X_resize)

        for i in range(0, 8):
            count+=1
            values.append(int_scale)
            filename = "{0}.jpg".format(count)
            filepath = save_path.format(filename)
            imageio.imwrite(filepath, translated_imgs[i])

        # Rotation
        rotated = rotate_images(X_resize, -90, 90, 20)

        for i in range(20):
            count+=1
            values.append(int_scale)
            filename = "{0}.jpg".format(count)
            filepath = save_path.format(filename)    
            imageio.imwrite(filepath, rotated[i])

        # Lighting
        count+=1
        values.append(int_scale)
        gaussian = add_gaussian_noise(X_resize)
        filename = "{0}.jpg".format(count)
        filepath = save_path.format(filename)
        imageio.imwrite(filepath, gaussian[0])

        
        print("---------number of images are {1}----------".format(int_scale, count))

        label = {"Value":values}
        label = pd.DataFrame(label)
        label.to_csv("label.csv", index=False)

        flag = False


print("# of all values",len(values))
print("Total Counts = ",count)





Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1321940422058105]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1322593688964844]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.132380723953247]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.132628083229065]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.13284170627594]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8

---------Iteration -1.0: number of images are 58----------


Lossy conversion from float32 to uint8. Range [0.0, 1.1330742835998535]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1330745220184326]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1331439018249512]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1329469680786133]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.133101224899292]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1332759857177734]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to ui

---------Iteration -0.9: number of images are 116----------


Lossy conversion from float32 to uint8. Range [0.0, 1.1330033540725708]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1330008506774902]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1326836347579956]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1332063674926758]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1330742835998535]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1330745220184326]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1331439018249512]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to u

...

---------Iteration 2.8: number of images are 2262----------


Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to u

---------Iteration 2.9: number of images are 2320----------


Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 1.1333333253860474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to u

---------Iteration 3.0: number of images are 2378----------
# of all values 2378
Total Counts =  2378
